In [1]:
suppressMessages(library("rwwa"))

In [2]:
gmst <- read.table("gmst.txt", col.names = c("year", "gmst"))
gmst$gmst <- gmst$gmst - gmst$gmst[gmst$year == 2025]

cov_2025 <- gmst[gmst$year == 2025,"gmst",drop = F]
cov_cf <- rbind("pi" = cov_2025 - 1.3, "2000" = cov_2025 - 0.65)

In [3]:
df <- read.csv("tx5x-monthly_era5_cities.csv")

df$year <- as.integer(substr(df$time,1,4))
df$month <- as.integer(substr(df$time,6,7))

In [4]:
# m_rates <- rep(c(1,1,1,1,1,3,2,1,1,1,1,1), 76)
# gmst_y <- rep(gmst$gmst[(gmst$year >= 1950) & (gmst$year <= 2025)], each = 12)
# tmean <- 25 + gmst_y * m_rates

# df <- data.frame(year = rep(1950:2025, each = 12),
#                  t2m = revd(12*76, loc = tmean),
#                  month = rep(1:12, 76))

In [6]:
jj_res <- function(m_jun, m_jul) {
    res_6 <- mdl_ests(m_jun, cov_f = cov_2025, cov_cf = cov_cf)
    res_7 <- mdl_ests(m_jul, cov_f = cov_2025, cov_cf = cov_cf)
    c(setNames(res_6, paste0("jun_", names(res_6))), setNames(res_7, paste0("jul_", names(res_7))),
      res_6[grepl("dI_abs", names(res_6))] - res_7[grepl("dI_abs", names(res_7))],
      res_6[grepl("alpha_gmst", names(res_6))] - res_7[grepl("alpha_gmst", names(res_7))])
}

In [93]:
for(cnm in  colnames(df)[2:(ncol(df)-2)]) {
    
    res_fnm <- paste0("compare-jun-july-change_tg5x_",cnm,"_era5.csv")
    if (file.exists(res_fnm)) next
    
    df_c <- merge(gmst, df[,c("year", "month", cnm)])
    
    df_jun <- df_c[df_c$month == 6,]
    df_jul <- df_c[df_c$month == 7,]
    
    mdl_jun <- tryCatch(fit_ns("gev", "shift", df_jun, varnm = cnm, covnm = "gmst", lower = F),
                        error = function(cond) {fit_ns("gev", "shift", df_jun[df_jun$year < 2025,], varnm = cnm, covnm = "gmst", lower = F, ev = df_jun[df_jun$year == 2025,cnm])})
    mdl_jul <- fit_ns("gev", "shift", df_jul, varnm = cnm, covnm = "gmst", lower = F)

    obs_res <- jj_res(mdl_jun, mdl_jul)
    
    write.csv(data.frame("year" = df_jun$year, 
                         "june" = ns_pars(mdl_jun, fixed_cov = df_jun[,"gmst",drop = F])$loc,
                         "july" = ns_pars(mdl_jul, fixed_cov = df_jun[,"gmst",drop = F])$loc),
                         gsub("compare-jun-july-change", "fitted-values-jj", res_fnm), row.names = F)
    
    # bootstrap whole years to get difference in trends
    set.seed(42)
    boot_res <- list()
    i <- 1
    f <- 0
    while(length(boot_res) < 1000) {
        ysamp <- sample(1:nrow(df_c[df_c$month == 6,]), replace = T)
        boot_df_jun <- df_jun[ysamp, ]
        boot_df_jul <- df_jul[ysamp, ]
        tryCatch({
            boot_jun <- refit(mdl_jun, new_data = boot_df_jun)
            boot_jul <- refit(mdl_jul, new_data = boot_df_jul)
            boot_res[[i]] <- jj_res(boot_jun, boot_jul)
            i <- i+1
        }, error = function(cond) {
            f <- f+1
            return(rep(NA, length(obs_res)))
        })
    }
    boot_res <- do.call("cbind",boot_res)
    
    write.csv(rbind("est" = obs_res, apply(boot_res,1,quantile,c(0.025,0.5,0.975), na.rm = T)), res_fnm)
    write.csv(boot_res, gsub("compare-", "full-bootstrap_", res_fnm))
}